<a href="https://colab.research.google.com/github/tvalli736/customer-support-ticket-triage/blob/main/Customer_Support_Triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Cell 1: Import necessary libraries
import pandas as pd
import re
from datetime import datetime, timedelta

# Cell 2: Simulate or load the dataset (replace with actual Kaggle CSV upload)
# In Colab, upload the CSV file and use pd.read_csv('/content/your_file.csv')
# For this example, I'm simulating a dataset with 10 rows.
data = {
    'ticket_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'customer_message': [
        "I can't log in to my account! Urgent help needed.",
        "My payment was charged twice. Refund please!",
        "Package not delivered yet. Where is it?",
        "App keeps crashing. Bug report.",
        "How do I reset my password?",
        "Payment failed during checkout.",
        "Delivery delayed by 3 days.",
        "Refund not processed after return.",
        "Login issues on mobile app.",
        "General inquiry about subscription."
    ],
    'timestamp': [
        '2023-10-01 10:00:00', '2023-10-01 11:00:00', '2023-10-01 12:00:00',
        '2023-10-01 13:00:00', '2023-10-01 14:00:00', '2023-10-01 15:00:00',
        '2023-10-01 16:00:00', '2023-10-01 17:00:00', '2023-10-01 18:00:00',
        '2023-10-01 19:00:00'
    ]
}
df = pd.DataFrame(data)
print("Dataset loaded successfully.")

# Cell 3: Explore the dataset
print("Dataset Shape (rows, columns):", df.shape)
print("\nColumns:", list(df.columns))
print("\nMissing Values:\n", df.isnull().sum())
print("\nSample Tickets (first 5):\n", df.head())

# Cell 4: Clean ticket messages
def clean_message(message):
    # Convert to lowercase
    message = message.lower()
    # Remove special characters and extra spaces
    message = re.sub(r'[^\w\s]', '', message)  # Remove punctuation
    message = re.sub(r'\s+', ' ', message).strip()  # Remove extra spaces
    return message

df['cleaned_message'] = df['customer_message'].apply(clean_message)
print("Messages cleaned. Sample cleaned messages:\n", df[['customer_message', 'cleaned_message']].head())

# Cell 5: Classify issues using rule-based logic
def classify_issue(message):
    if 'payment' in message or 'charged' in message:
        return 'PAYMENT'
    elif 'log' in message or 'login' in message or 'password' in message:
        return 'LOGIN'
    elif 'deliver' in message or 'package' in message:
        return 'DELIVERY'
    elif 'refund' in message:
        return 'REFUND'
    elif 'bug' in message or 'crash' in message:
        return 'BUG'
    else:
        return 'GENERAL'

df['issue_type'] = df['cleaned_message'].apply(classify_issue)
print("Issues classified. Sample:\n", df[['cleaned_message', 'issue_type']].head())

# Cell 6: Assign priority levels based on keywords
def assign_priority(message):
    urgent_keywords = ['urgent', 'help needed', 'failed', 'delayed']
    high_keywords = ['twice', 'not processed', 'crashing']
    if any(word in message for word in urgent_keywords):
        return 'P0'  # Highest priority
    elif any(word in message for word in high_keywords):
        return 'P1'
    elif 'inquiry' in message or 'general' in message:
        return 'P3'  # Lowest
    else:
        return 'P2'  # Medium

df['priority'] = df['cleaned_message'].apply(assign_priority)
print("Priorities assigned. Sample:\n", df[['cleaned_message', 'priority']].head())

# Cell 7: Calculate SLA hours and due time
sla_hours = {'P0': 1, 'P1': 4, 'P2': 24, 'P3': 72}
df['sla_hours'] = df['priority'].map(sla_hours)
df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert to datetime
df['due_time'] = df['timestamp'] + pd.to_timedelta(df['sla_hours'], unit='h')
print("SLA calculated. Sample:\n", df[['priority', 'sla_hours', 'due_time']].head())

# Cell 8: Generate support manager report and export final CSV
# Summary report
total_tickets = len(df)
priority_counts = df['priority'].value_counts()
issue_counts = df['issue_type'].value_counts()
overdue_tickets = df[df['due_time'] < datetime.now()]  # Assuming current time; in real-time, use pd.Timestamp.now()

print("=== Support Manager Report ===")
print(f"Total Tickets: {total_tickets}")
print(f"Priority Breakdown:\n{priority_counts}")
print(f"Issue Type Breakdown:\n{issue_counts}")
print(f"Overdue Tickets: {len(overdue_tickets)}")
if not overdue_tickets.empty:
    print("Overdue Ticket IDs:", list(overdue_tickets['ticket_id']))

# Export final CSV
output_df = df[['ticket_id', 'customer_message', 'issue_type', 'priority', 'sla_hours', 'due_time']]
output_df.to_csv('/content/triage_output.csv', index=False)
print("Final CSV exported to /content/triage_output.csv")
print("Sample of final output:\n", output_df.head())

Dataset loaded successfully.
Dataset Shape (rows, columns): (10, 3)

Columns: ['ticket_id', 'customer_message', 'timestamp']

Missing Values:
 ticket_id           0
customer_message    0
timestamp           0
dtype: int64

Sample Tickets (first 5):
    ticket_id                                   customer_message  \
0          1  I can't log in to my account! Urgent help needed.   
1          2       My payment was charged twice. Refund please!   
2          3            Package not delivered yet. Where is it?   
3          4                    App keeps crashing. Bug report.   
4          5                        How do I reset my password?   

             timestamp  
0  2023-10-01 10:00:00  
1  2023-10-01 11:00:00  
2  2023-10-01 12:00:00  
3  2023-10-01 13:00:00  
4  2023-10-01 14:00:00  
Messages cleaned. Sample cleaned messages:
                                     customer_message  \
0  I can't log in to my account! Urgent help needed.   
1       My payment was charged twice. Ref